In [1]:
from bggcomplex import BGGComplex
from fast_module import FastLieAlgebraCompositeModule, FastModuleFactory, BGGCohomology
import numpy as np
%load_ext cython

In [2]:
for d in ['A1','A2','A3','B2','G2','A4','B3','C3']:
    BGG = BGGComplex(d)
    f_factory = FastModuleFactory(BGG.LA)
    component_dic = {'b':f_factory.build_component('b','ad'),'u':f_factory.build_component('u','coad')
                     ,'b+':f_factory.build_component('b+','coad'),'n':f_factory.build_component('n','ad')}
    components = [[("n",1,'wedge'),("n",1,'wedge')]]

    module = FastLieAlgebraCompositeModule(f_factory, components, component_dic)

    cohom = BGGCohomology(BGG, module)
    for i in range(10):
        cohom.cohomology_LaTeX(i,only_non_zero=True,complex_string=d+r'/B, \mathfrak n\otimes \mathfrak n')

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [30]:
len(component_dic['b'].basis)

14

In [40]:
d='A4'
BGG = BGGComplex(d)
f_factory = FastModuleFactory(BGG.LA)
component_dic = {'b':f_factory.build_component('b','ad'),'u':f_factory.build_component('u','coad')
                 ,'b+':f_factory.build_component('b+','coad'),'n':f_factory.build_component('n','ad')}
components = [[("u",2,'wedge')]]

module = FastLieAlgebraCompositeModule(f_factory, components, component_dic)

cohom = BGGCohomology(BGG, module)
for i in range(10):
    cohom.cohomology_LaTeX(i,only_non_zero=True,complex_string=d+r'/B, \wedge^2\mathfrak u')

<IPython.core.display.Math object>

In [66]:


weight_mat = np.array([s[1] for s in sorted(module.weight_dic.items(), key=lambda t: t[0])], dtype=np.int64)

tot_weight = np.sum(weight_mat[d_sum_comp],axis=1)
argsort = np.lexsort(np.transpose(tot_weight))
split_dic = {}
last_weight = tot_weight[0]
current_inds = []
for i in argsort:
    if np.all(np.equal(tot_weight[i],last_weight)):
        current_inds.append(i)
    else:
        split_dic[tuple(last_weight)]=d_sum_comp[current_inds]
        current_inds=[i]
        last_weight=tot_weight[i]
return split_dic

In [79]:
split_dic

{(0, 0, 1, 1): array([[16, 19]], dtype=int32),
 (0, 0, 1, 2): array([[10, 19]], dtype=int32),
 (0, 0, 2, 1): array([[10, 16]], dtype=int32),
 (0, 1, 0, 1): array([[19, 20]], dtype=int32),
 (0, 1, 1, 0): array([[16, 20]], dtype=int32),
 (0, 1, 1, 1): array([[10, 20],
        [12, 19]], dtype=int32),
 (0, 1, 1, 2): array([[18, 19]], dtype=int32),
 (0, 1, 2, 0): array([[12, 16]], dtype=int32),
 (0, 1, 2, 1): array([[10, 12],
        [16, 18]], dtype=int32),
 (0, 1, 2, 2): array([[10, 18]], dtype=int32),
 (0, 2, 1, 0): array([[12, 20]], dtype=int32),
 (0, 2, 1, 1): array([[18, 20]], dtype=int32),
 (0, 2, 2, 1): array([[12, 18]], dtype=int32),
 (1, 0, 0, 1): array([[15, 19]], dtype=int32),
 (1, 0, 1, 0): array([[15, 16]], dtype=int32),
 (1, 0, 1, 1): array([[10, 15]], dtype=int32),
 (1, 1, 0, 0): array([[15, 20]], dtype=int32),
 (1, 1, 0, 1): array([[19, 23]], dtype=int32),
 (1, 1, 1, 0): array([[12, 15],
        [16, 23]], dtype=int32),
 (1, 1, 1, 1): array([[10, 23],
        [11, 19],
   

In [80]:
module.compute_weight_components(d_sum_comp)

{(0, 0, 1, 1): array([[16, 19]], dtype=int32),
 (0, 0, 1, 2): array([[10, 19]], dtype=int32),
 (0, 0, 2, 1): array([[10, 16]], dtype=int32),
 (0, 1, 0, 1): array([[19, 20]], dtype=int32),
 (0, 1, 1, 0): array([[16, 20]], dtype=int32),
 (0, 1, 1, 1): array([[10, 20],
        [12, 19]], dtype=int32),
 (0, 1, 1, 2): array([[18, 19]], dtype=int32),
 (0, 1, 2, 0): array([[12, 16]], dtype=int32),
 (0, 1, 2, 1): array([[10, 12],
        [16, 18]], dtype=int32),
 (0, 1, 2, 2): array([[10, 18]], dtype=int32),
 (0, 2, 1, 0): array([[12, 20]], dtype=int32),
 (0, 2, 1, 1): array([[18, 20]], dtype=int32),
 (0, 2, 2, 1): array([[12, 18]], dtype=int32),
 (1, 0, 0, 1): array([[15, 19]], dtype=int32),
 (1, 0, 1, 0): array([[15, 16]], dtype=int32),
 (1, 0, 1, 1): array([[10, 15]], dtype=int32),
 (1, 1, 0, 0): array([[15, 20]], dtype=int32),
 (1, 1, 0, 1): array([[19, 23]], dtype=int32),
 (1, 1, 1, 0): array([[12, 15],
        [16, 23]], dtype=int32),
 (1, 1, 1, 1): array([[10, 23],
        [11, 19],
   